In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from sklearn import preprocessing as pp
from sklearn import cross_validation as cv
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.svm import SVC
from sklearn import tree
from sklearn.cross_validation import cross_val_score

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import keras

Using TensorFlow backend.


In [4]:
keras.__version__

'2.1.1'

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD

### Strip the input data 

In [6]:
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
print(train.shape, test.shape)

(891, 11) (418, 10)


In [7]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
cols_to_drop = ["Name", "Ticket", "Cabin"]

In [9]:
train.drop(cols_to_drop, axis= 1, inplace=True)
test.drop(cols_to_drop, axis= 1, inplace=True)

In [10]:
gender_mapping = {'female':0, 'male':1}
train['Sex'] = train['Sex'].map(gender_mapping)
test['Sex'] = test['Sex'].map(gender_mapping)

In [11]:
embarked_mapping = {'S':0, 'C':1, 'Q': 2}
train['Embarked'] = train['Embarked'].map(embarked_mapping)
test['Embarked'] = test['Embarked'].map(embarked_mapping)

In [12]:
test.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000,418.000000
mean,2.265550,0.636364,30.272590,0.447368,0.392344,35.627188,0.464115
std,0.841838,0.481622,14.181209,0.896760,0.981429,55.907576,0.685516
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,3.000000,1.000000,27.000000,0.000000,0.000000,14.454200,0.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.500000,1.000000
max,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


### Dealing with missing values

In [13]:
test.Age.fillna(inplace=True, value=test.Age.mean())
test.Fare.fillna(inplace=True, value=test.Fare.mean())

In [14]:
test.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.636364,30.272590,0.447368,0.392344,35.627188,0.464115
std,0.841838,0.481622,12.634534,0.896760,0.981429,55.840500,0.685516
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,23.000000,0.000000,0.000000,7.895800,0.000000
50%,3.000000,1.000000,30.272590,0.000000,0.000000,14.454200,0.000000
75%,3.000000,1.000000,35.750000,1.000000,0.000000,31.500000,1.000000
max,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


In [15]:
train.dropna(inplace=True)
test.dropna(inplace=True)
print(train.shape, test.shape)

(712, 8) (418, 7)


In [16]:
y = train.Survived
train.drop("Survived", axis= 1, inplace=True)
X = train
X_final = test

### Define the parameters

In [17]:
batch_size = 16    #we have only 712 elements
num_classes = 2    #Survived or not
number_of_epochs=200

In [18]:
test_size = 0.2
random_state = 4


### Normalize the data

In [19]:
X_max = X.append(X_final).max().astype(np.float64)
print(X_max)

Pclass        3.0000
Sex           1.0000
Age          80.0000
SibSp         8.0000
Parch         9.0000
Fare        512.3292
Embarked      2.0000
dtype: float64


In [20]:
X_scaled = X/X.max().astype(np.float64)
X_final  = X_final/X.max().astype(np.float64)

In [21]:
X_scaled.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,1.000000,1.0,0.2750,0.2,0.0,0.014151,0.0
2,0.333333,0.0,0.4750,0.2,0.0,0.139136,0.5
3,1.000000,0.0,0.3250,0.0,0.0,0.015469,0.0
4,0.333333,0.0,0.4375,0.2,0.0,0.103644,0.0
5,1.000000,1.0,0.4375,0.0,0.0,0.015713,0.0


### Split data into train and test

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size= test_size, random_state = random_state)

In [23]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(type(X_train), type(X_test), type(y_train), type(y_test))

(569, 7) (143, 7) (569,) (143,)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [24]:
y_train = keras.utils.to_categorical(np.array(y_train), num_classes)
y_test = keras.utils.to_categorical(np.array(y_test), num_classes)

In [25]:
X_train = (np.matrix(X_train))
X_test = (np.matrix(X_test))
X_final = (np.matrix(X_final))

In [26]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(type(X_train), type(X_test), type(y_train), type(y_test))

(569, 7) (143, 7) (569, 2) (143, 2)
<class 'numpy.matrixlib.defmatrix.matrix'> <class 'numpy.matrixlib.defmatrix.matrix'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


### Build the Architecture

In [27]:
#Instatiaate a model
model = Sequential()

In [28]:
number_of_featrures = 7
number_of_nodes = 70

In [29]:
#Add later to it
model.add(Dense(70, activation="relu", input_shape=(7,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(2, activation='softmax'))

In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 70)                560       
_________________________________________________________________
dense_2 (Dense)              (None, 70)                4970      
_________________________________________________________________
dense_3 (Dense)              (None, 70)                4970      
_________________________________________________________________
dense_4 (Dense)              (None, 70)                4970      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 142       
Total params: 15,612
Trainable params: 15,612
Non-trainable params: 0
_________________________________________________________________


In [31]:
sgd = SGD(lr= 0.01)

In [32]:
#Complie the model for running
model.compile(loss='categorical_crossentropy',
             optimizer = sgd,
             metrics =['accuracy'])

### Train the model 

In [33]:
model.fit(X_train, y_train, batch_size= batch_size, epochs=number_of_epochs, verbose=1, validation_data=(X_test, y_test))

Train on 569 samples, validate on 143 samples
Epoch 1/200
569/569 [==============================] - 0s 375us/step - loss: 0.6877 - acc: 0.5905 - val_loss: 0.6632 - val_acc: 0.6364
Epoch 2/200
569/569 [==============================] - 0s 186us/step - loss: 0.6544 - acc: 0.5975 - val_loss: 0.6408 - val_acc: 0.5944
Epoch 3/200
569/569 [==============================] - 0s 189us/step - loss: 0.6366 - acc: 0.6063 - val_loss: 0.6240 - val_acc: 0.5944
Epoch 4/200
569/569 [==============================] - 0s 181us/step - loss: 0.6214 - acc: 0.6186 - val_loss: 0.6089 - val_acc: 0.6713
Epoch 5/200
569/569 [==============================] - 0s 190us/step - loss: 0.6065 - acc: 0.7100 - val_loss: 0.5942 - val_acc: 0.7622
Epoch 6/200
569/569 [==============================] - 0s 187us/step - loss: 0.5917 - acc: 0.7786 - val_loss: 0.5787 - val_acc: 0.7762
Epoch 7/200
569/569 [==============================] - 0s 285us/step - loss: 0.5763 - acc: 0.7909 - val_loss: 0.5644 - val_acc: 0.7832
Epoch 8/2

Epoch 61/200
569/569 [==============================] - 0s 191us/step - loss: 0.4432 - acc: 0.8067 - val_loss: 0.4202 - val_acc: 0.8252
Epoch 62/200
569/569 [==============================] - 0s 281us/step - loss: 0.4434 - acc: 0.7961 - val_loss: 0.4191 - val_acc: 0.7972
Epoch 63/200
569/569 [==============================] - 0s 183us/step - loss: 0.4418 - acc: 0.8084 - val_loss: 0.4171 - val_acc: 0.7972
Epoch 64/200
569/569 [==============================] - 0s 186us/step - loss: 0.4420 - acc: 0.8032 - val_loss: 0.4158 - val_acc: 0.8182
Epoch 65/200
569/569 [==============================] - 0s 185us/step - loss: 0.4409 - acc: 0.8067 - val_loss: 0.4214 - val_acc: 0.7902
Epoch 66/200
569/569 [==============================] - 0s 188us/step - loss: 0.4425 - acc: 0.8014 - val_loss: 0.4147 - val_acc: 0.8182
Epoch 67/200
569/569 [==============================] - 0s 184us/step - loss: 0.4404 - acc: 0.7979 - val_loss: 0.4149 - val_acc: 0.8042
Epoch 68/200
569/569 [==========================

569/569 [==============================] - 0s 186us/step - loss: 0.4259 - acc: 0.8155 - val_loss: 0.3988 - val_acc: 0.8182
Epoch 122/200
569/569 [==============================] - 0s 190us/step - loss: 0.4245 - acc: 0.8172 - val_loss: 0.4012 - val_acc: 0.8252
Epoch 123/200
569/569 [==============================] - 0s 207us/step - loss: 0.4264 - acc: 0.8225 - val_loss: 0.4083 - val_acc: 0.8112
Epoch 124/200
569/569 [==============================] - 0s 265us/step - loss: 0.4243 - acc: 0.8190 - val_loss: 0.4073 - val_acc: 0.8112
Epoch 125/200
569/569 [==============================] - 0s 187us/step - loss: 0.4256 - acc: 0.8207 - val_loss: 0.4021 - val_acc: 0.8182
Epoch 126/200
569/569 [==============================] - 0s 190us/step - loss: 0.4245 - acc: 0.8260 - val_loss: 0.4088 - val_acc: 0.8252
Epoch 127/200
569/569 [==============================] - 0s 187us/step - loss: 0.4230 - acc: 0.8190 - val_loss: 0.4103 - val_acc: 0.8042
Epoch 128/200
569/569 [==============================] 

Epoch 181/200
569/569 [==============================] - 0s 282us/step - loss: 0.4058 - acc: 0.8278 - val_loss: 0.3999 - val_acc: 0.8462
Epoch 182/200
569/569 [==============================] - 0s 190us/step - loss: 0.4063 - acc: 0.8190 - val_loss: 0.4091 - val_acc: 0.8042
Epoch 183/200
569/569 [==============================] - 0s 185us/step - loss: 0.4047 - acc: 0.8383 - val_loss: 0.4161 - val_acc: 0.8112
Epoch 184/200
569/569 [==============================] - 0s 186us/step - loss: 0.4068 - acc: 0.8260 - val_loss: 0.3987 - val_acc: 0.8182
Epoch 185/200
569/569 [==============================] - 0s 187us/step - loss: 0.4056 - acc: 0.8313 - val_loss: 0.4148 - val_acc: 0.8042
Epoch 186/200
569/569 [==============================] - 0s 283us/step - loss: 0.4034 - acc: 0.8383 - val_loss: 0.3952 - val_acc: 0.8392
Epoch 187/200
569/569 [==============================] - 0s 188us/step - loss: 0.4068 - acc: 0.8207 - val_loss: 0.3989 - val_acc: 0.8252
Epoch 188/200
569/569 [==================

In [34]:
score = model.evaluate(X_test, y_test, verbose=0)

In [35]:
print('Test Loss', score[0])
print('Test Accuracy', score[1])

Test Loss 0.397956008678
Test Accuracy 0.818181819015


## Retrain the model

### Re build the architecure

In [36]:
X_scaled = np.matrix(X_scaled)
y_scaled = keras.utils.to_categorical(np.array(y), num_classes)
print(type(X_scaled), type(y_scaled))

<class 'numpy.matrixlib.defmatrix.matrix'> <class 'numpy.ndarray'>


In [37]:
#Instatiaate a model
model = Sequential()

In [38]:
#Add later to it
model.add(Dense(70, activation="relu", input_shape=(7,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(2, activation='softmax'))

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 70)                560       
_________________________________________________________________
dense_7 (Dense)              (None, 70)                4970      
_________________________________________________________________
dense_8 (Dense)              (None, 70)                4970      
_________________________________________________________________
dense_9 (Dense)              (None, 70)                4970      
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 142       
Total params: 15,612
Trainable params: 15,612
Non-trainable params: 0
_________________________________________________________________


In [40]:
sgd = SGD(lr= 0.01)

#Complie the model for running
model.compile(loss='categorical_crossentropy',
             optimizer = sgd,
             metrics =['accuracy'])

In [41]:
number_of_epochs = number_of_epochs + 40
print(number_of_epochs)

240


In [42]:
model.fit(X_scaled, y_scaled, batch_size= batch_size, epochs=number_of_epochs, verbose=1)

Epoch 1/240
712/712 [==============================] - 0s 322us/step - loss: 0.6745 - acc: 0.6110
Epoch 2/240
712/712 [==============================] - 0s 166us/step - loss: 0.6409 - acc: 0.6096
Epoch 3/240
712/712 [==============================] - 0s 152us/step - loss: 0.6155 - acc: 0.6854
Epoch 4/240
712/712 [==============================] - 0s 238us/step - loss: 0.5931 - acc: 0.7739
Epoch 5/240
712/712 [==============================] - 0s 155us/step - loss: 0.5738 - acc: 0.7781
Epoch 6/240
712/712 [==============================] - 0s 158us/step - loss: 0.5560 - acc: 0.7795
Epoch 7/240
712/712 [==============================] - 0s 241us/step - loss: 0.5418 - acc: 0.7795
Epoch 8/240
712/712 [==============================] - 0s 161us/step - loss: 0.5302 - acc: 0.7795
Epoch 9/240
712/712 [==============================] - 0s 159us/step - loss: 0.5218 - acc: 0.7795
Epoch 10/240
712/712 [==============================] - 0s 240us/step - loss: 0.5140 - acc: 0.7795
Epoch 11/240
712/71

712/712 [==============================] - 0s 160us/step - loss: 0.4118 - acc: 0.8343
Epoch 81/240
712/712 [==============================] - 0s 156us/step - loss: 0.4124 - acc: 0.8244
Epoch 82/240
712/712 [==============================] - 0s 240us/step - loss: 0.4144 - acc: 0.8188
Epoch 83/240
712/712 [==============================] - 0s 160us/step - loss: 0.4141 - acc: 0.8174
Epoch 84/240
712/712 [==============================] - 0s 158us/step - loss: 0.4124 - acc: 0.8230
Epoch 85/240
712/712 [==============================] - 0s 239us/step - loss: 0.4132 - acc: 0.8244
Epoch 86/240
712/712 [==============================] - 0s 160us/step - loss: 0.4101 - acc: 0.8272
Epoch 87/240
712/712 [==============================] - 0s 157us/step - loss: 0.4064 - acc: 0.8385
Epoch 88/240
712/712 [==============================] - 0s 242us/step - loss: 0.4118 - acc: 0.8272
Epoch 89/240
712/712 [==============================] - 0s 157us/step - loss: 0.4100 - acc: 0.8301
Epoch 90/240
712/712 [=

712/712 [==============================] - 0s 158us/step - loss: 0.3936 - acc: 0.8343
Epoch 161/240
712/712 [==============================] - 0s 242us/step - loss: 0.3974 - acc: 0.8315
Epoch 162/240
712/712 [==============================] - 0s 165us/step - loss: 0.3933 - acc: 0.8413
Epoch 163/240
712/712 [==============================] - 0s 160us/step - loss: 0.3915 - acc: 0.8483
Epoch 164/240
712/712 [==============================] - 0s 229us/step - loss: 0.3947 - acc: 0.8385
Epoch 165/240
712/712 [==============================] - 0s 160us/step - loss: 0.3939 - acc: 0.8371
Epoch 166/240
712/712 [==============================] - 0s 158us/step - loss: 0.3926 - acc: 0.8258
Epoch 167/240
712/712 [==============================] - 0s 240us/step - loss: 0.3931 - acc: 0.8357 1s - loss: 0.2569 - acc: 0.93
Epoch 168/240
712/712 [==============================] - 0s 161us/step - loss: 0.3913 - acc: 0.8413 0s - loss: 0.3865 - acc: 0.837
Epoch 169/240
712/712 [==============================

712/712 [==============================] - 0s 159us/step - loss: 0.3849 - acc: 0.8357
Epoch 239/240
712/712 [==============================] - 0s 158us/step - loss: 0.3823 - acc: 0.8427
Epoch 240/240
712/712 [==============================] - 0s 240us/step - loss: 0.3811 - acc: 0.8357


In [43]:
y_final_pred = model.predict_classes(X_final)

In [44]:
y_final_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

### Write to file

In [45]:
file_name = "titanic_results_10_nn.csv"
override_writing = False

In [46]:
y_final = pd.DataFrame()

In [47]:
y_final["PassengerId"] = test.index
y_final["Survived"] = pd.Series(y_final_pred)

In [48]:
y_final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [49]:

y_final.to_csv(file_name, encoding='utf-8', index=False)